# Fine-tuning Mistral-7B con QLoRA

In [ ]:
!pip install transformers datasets peft bitsandbytes accelerate

In [ ]:
!git clone https://github.com/your-user/bank-qa-finetune-hf.git && cd bank-qa-finetune-hf

In [ ]:
!python train.py